Load Zinc-250k dataset in SMILES format

TODO rather download in script?

https://github.com/aspuru-guzik-group/chemical_vae 

From https://github.com/aspuru-guzik-group/chemical_vae/blob/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv

https://github.com/chainer/chainer-chemistry/blob/master/chainer_chemistry/datasets/zinc.py

In [2]:
import pandas as pd
import numpy as np
import data.mol_utils as mu

In [3]:
df_raw = pd.read_csv("data/250k_rndm_zinc_drugs_clean_3.csv")
df_raw.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


Preprocessing

- subset of 35 different characters for ZINC  TODO Which ones?
- Shorter strings were padded with spaces, total length 120
- canonicalized SMILES for training

Model input:
- one-hot encoding (e.g. tutorial https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html#turning-names-into-tensors)

In [4]:
def remove_newline(s):
    return s.strip()

def pad(s):
    return s.ljust(120)

# TODO canonicalize strings

remove_newline(df_raw['smiles'].iloc[0])
pad(remove_newline(df_raw['smiles'].iloc[0]))

'CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1                                                                                 '

In [5]:
"asldfkj".ljust(10)

'asldfkj   '

Gaussian Process Prediction target:
5× QED − SAS
started with molecules from the ZINC dataset that were in the bottom 10% percentile of our objective.

# Using original code

In [6]:
smiles, Y_reg = mu.load_smiles_and_data_df('data/250k_rndm_zinc_drugs_clean_3.csv', max_len=120, reg_tasks=['logP', 'qed', 'SAS'])# , normalize_out='data/reg_normalise.csv')

In [7]:
# mu.smiles_to_hot_filter(smiles)
MAX_LEN = 120
CHARS = mu.smiles2one_hot_chars(smiles, MAX_LEN)

In [8]:
NCHARS = len(CHARS)
CHAR_INDICES = dict((c, i) for i, c in enumerate(CHARS))

In [9]:
# TODO: Canonicalise
X = mu.smiles_to_hot(smiles, max_len=MAX_LEN, padding='right', char_indices=CHAR_INDICES, nchars=NCHARS)

In [17]:
X_bool = X.astype(np.bool)

In [19]:
X_bool.size

1047711000

In [12]:
X.size * X.itemsize

4190844000

In [59]:
print("data size: ", X.shape[0])

data size:  249455


In [20]:
np.savez_compressed('data/train_compressed_bool', X_bool)

In [61]:
assert Y_reg.shape[0] == X.shape[0]

In [64]:
np.save('data/Y_reg', Y_reg)

# TMP

In [33]:
tmp = 'COc1ccc(cc1)S(=O)(=O)NCc2ccccc2OC'

def pad_smile(string, max_len, padding='right'):
    if len(string) <= max_len:
        if padding == 'right':
            return string + " " * (max_len - len(string))
        elif padding == 'left':
            return " " * (max_len - len(string)) + string
        elif padding == 'none':
            return string

def smiles_to_hot(smiles, max_len, padding, char_indices, nchars):
    smiles = [pad_smile(i, max_len, padding)
              for i in smiles if pad_smile(i, max_len, padding)]

    X = np.zeros((len(smiles), max_len, nchars), dtype=np.float32)
    print(X.shape)

    for i, smile in enumerate(smiles):
        for t, char in enumerate(smile):
            try:
                X[i, t, char_indices[char]] = 1
            except KeyError as e:
                print("ERROR: Check chars file. Bad SMILES:", smile)
                raise e
    return X

def smiles2one_hot_chars(smi_list, max_len):
    # get all the characters
    char_lists = [list(smi) for smi in smi_list]
    chars = list(set([char for sub_list in char_lists for char in sub_list]))
    chars.append(' ')

    return chars

char_indices = dict(zip(smiles2one_hot_chars([tmp], 120), range(0, len(tmp))))
print(char_indices)
example_input = smiles_to_hot([tmp], 120, 'right', char_indices, len(char_indices))
print(example_input)

{'2': 0, 'C': 1, '(': 2, '1': 3, 'N': 4, ')': 5, 'S': 6, '=': 7, 'c': 8, 'O': 9, ' ': 10}
(1, 120, 11)
[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]]
